# MONAI Auto3DSeg Hyper-parameter Optimization with Optuna

This notebook provides an example to perform hype-parameter optimization(HPO) on learning rate with grid search method for spleen segmentation using Optuna.

Note: if you have used other notebooks under `auto3dseg`, for examples: 
- `auto_runner.ipynb`
- `auto3dseg_autorunner_ref_api.ipynb`
- `auto3dseg_hello_world.ipynb`
- `hpo_nni.ipynb`

You may have already generated the algorithm templates in MONAI bundle formats (hint: find them in the working directory). 

Please feel free to skip step 1-5 if the bundles are already generated.

## 1. Import libraries for HPO and pipelines

In [1]:
import os
import optuna
import torch
import yaml

from functools import partial

from monai.apps import download_and_extract
from monai.apps.auto3dseg import BundleGen, DataAnalyzer, OptunaGen
from monai.apps.auto3dseg.utils import export_bundle_algo_history, import_bundle_algo_history
from monai.bundle.config_parser import ConfigParser

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Define experiment file pathes

In [2]:
# Dataset pathes
data_root = "./"
msd_task = "Task09_Spleen"
dataroot = os.path.join(data_root, msd_task)

# User created files
datalist_file = os.path.join("..", "tasks", "msd", msd_task, "msd_" + msd_task.lower() + "_folds.json")
input_yaml = './input.yaml'

# Experiment setup
test_path = "./"
work_dir = os.path.join(test_path, "hpo_optuna_work_dir")
optuna_dir = './optuna_learningrate_grid'
da_output_yaml = os.path.join(work_dir, "datastats.yaml")
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# Algorithm selected to do HPO. Refer to bundle history for the mapping between
# algorithm name and index
selected_algorithm_index = 0

## 3. Download one of MSD datasets

In [3]:
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"
compressed_file = os.path.join(data_root, msd_task + ".tar")
if not os.path.exists(dataroot):
    os.makedirs(dataroot)
    download_and_extract(resource, compressed_file, data_root)

## 4. Generate input yaml and datafolds yaml. (User should generate their own)

In [4]:
input_dict = {
    "name": msd_task,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": datalist_file,
    "dataroot": dataroot,
    "multigpu": True,
    "class_names": ["val_acc_pz", "val_acc_tz"]
}

with open(input_yaml, 'w') as f:
    yaml.dump(input_dict, f)


## 5. Create Bundle Generators


In [5]:

cfg = ConfigParser.load_config_file(input_yaml)

# data analysis
if not os.path.exists(da_output_yaml):
    da = DataAnalyzer(datalist_file, dataroot, output_path=da_output_yaml)
    da.get_all_case_stats()

# algorithm generation
bundle_generator = BundleGen(
    algo_path=work_dir,
    data_stats_filename=da_output_yaml,
    data_src_cfg_name=input_yaml,
)

bundle_generator.generate(work_dir, num_fold=5)
history = bundle_generator.get_history()
export_bundle_algo_history(history)

100%|██████████| 40/40 [01:33<00:00,  2.34s/it]

2022-09-28 17:08:51,151 - WARNING - Data is not completely uniform. MONAI transforms may provide unexpected result



algo_templates.tar.gz: 100%|██████████| 280k/280k [00:00<00:00, 341kB/s]  

2022-09-28 17:08:52,165 - INFO - Downloaded: /tmp/tmpwptjcv79/algo_templates.tar.gz
2022-09-28 17:08:52,165 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmpwptjcv79/algo_templates.tar.gz.
2022-09-28 17:08:52,166 - INFO - Writing into directory: ./hpo_optuna_work_dir.


2022-09-28 17:08:59,605 - INFO - ./hpo_optuna_work_dir/segresnet2d_0
2022-09-28 17:09:01,295 - INFO - ./hpo_optuna_work_dir/segresnet2d_1
2022-09-28 17:09:03,774 - INFO - ./hpo_optuna_work_dir/segresnet2d_2
2022-09-28 17:09:04,496 - INFO - ./hpo_optuna_work_dir/segresnet2d_3
2022-09-28 17:09:05,354 - INFO - ./hpo_optuna_work_dir/segresnet2d_4
2022-09-28 17:09:06,206 - INFO - ./hpo_optuna_work_dir/dints_0
2022-09-28 17:09:07,010 - INFO - ./hpo_optuna_work_dir/dints_1
2022-09-28 17:09:09,408 - INFO - ./hpo_optuna_work_dir/dints_2
2022-09-28 17:09:12,231 - INFO - ./hpo_optuna_work_dir/dints_3
2022-09-28 17:09:14,473 - INFO - ./hpo_optuna_work_dir/dints_4
2022-09-28 17:09:15,362 - INFO - ./hpo_optuna_work_dir/swinunetr_0
2022-09-28 17:09:16,713 - INFO - ./hpo_optuna_work_dir/swinunetr_1
2022-09-28 17:09:20,002 - INFO - ./hpo_optuna_work_dir/swinunetr_2
2022-09-28 17:09:21,317 - INFO - ./hpo_optuna_work_dir/swinunetr_3
2022-09-28 17:09:23,820 - INFO - ./hpo_optuna_work_dir/swinunetr_4
2022-

## 6. Create Algo object from bundle_generator history

In [6]:
# you can get history from bundle_generator. It can also be acquired by reading bundles saved on disk

history = bundle_generator.get_history()
if len(history) == 0:
    history = import_bundle_algo_history(work_dir, only_trained=False)

algo_dict = history[selected_algorithm_index]
algo_name = list(algo_dict.keys())[selected_algorithm_index]
algo = algo_dict[algo_name]


In [7]:
# "override_params" is used to update algorithm hyperparameters 
# like num_epochs, which are not in the HPO search space. We set num_epochs=2
# to shorten the training time as an example

max_epochs = 2

# safeguard to ensure max_epochs is greater or equal to 2
max_epochs = max(max_epochs, 2)

num_gpus = 1 if "multigpu" in input_dict and not input_dict["multigpu"] else torch.cuda.device_count()

num_epoch = max_epochs
num_images_per_batch = 2
n_data = 24  # total is 30 images, hold out one set (6 images) for cross fold val.
n_iter = int(num_epoch * n_data / num_images_per_batch / num_gpus)
n_iter_val = int(n_iter / 2)

override_param = {
    "num_iterations": n_iter,
    "num_iterations_per_validation": n_iter_val,
    "num_images_per_batch": num_images_per_batch,
    "num_epochs": num_epoch,
    "num_warmup_iterations": n_iter_val,
}

## 7. Create Optuna Generator class and overwrite get_hyperparameters() function

In [8]:
class OptunaGenLearningRate(OptunaGen):
    def get_hyperparameters(self):
        return {'learning_rate': self.trial.suggest_float("learning_rate", 0.00001, 0.1)}


optuna_gen = OptunaGenLearningRate(algo=algo, params=override_param)

2022-09-28 17:09:37,228 - INFO - ./hpo_optuna_work_dir/segresnet2d_0_override


## 8. Run Optuna optimization (with grid search)

In [9]:
search_space = {'learning_rate': [0.0001, 0.001, 0.01, 0.1]}
study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space), direction='maximize')
study.optimize(partial(optuna_gen, obj_filename=optuna_gen.get_obj_filename(), output_folder=optuna_dir), n_trials=2)
print("Best value: {} (params: {})\n".format(study.best_value, study.best_params))

[I 2022-09-28 17:09:37,486] A new study created in memory with name: no-name-a34b729c-9f06-4439-b528-5199836c43d3


2022-09-28 17:09:39,227 - INFO - ./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001
2022-09-28 17:09:39,241 - INFO - Launching: python ./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001/scripts/train.py run --config_file='./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001/configs/network.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001/configs/transforms_validate.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001/configs/transforms_infer.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001/configs/hyper_parameters.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001/configs/transforms_train.yaml' --learning_rate=0.001
2022-09-28 17:12:50,088 - INFO - CompletedProcess(args=['python', './optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.001/scripts/train.py', 'run', "--config_file='./optuna_learningrate_grid/segresnet

[I 2022-09-28 17:12:50,109] Trial 0 finished with value: 0.012783133424818516 and parameters: {'learning_rate': 0.001}. Best is trial 0 with value: 0.012783133424818516.


2022-09-28 17:12:50,835 - INFO - ./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1
2022-09-28 17:12:50,854 - INFO - Launching: python ./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1/scripts/train.py run --config_file='./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1/configs/network.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1/configs/transforms_validate.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1/configs/transforms_infer.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1/configs/hyper_parameters.yaml','./optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1/configs/transforms_train.yaml' --learning_rate=0.1
2022-09-28 17:15:58,897 - INFO - CompletedProcess(args=['python', './optuna_learningrate_grid/segresnet2d_0_override_learning_rate_0.1/scripts/train.py', 'run', "--config_file='./optuna_learningrate_grid/segresnet2d_0_override_lear

[I 2022-09-28 17:15:59,029] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 0.1}. Best is trial 0 with value: 0.012783133424818516.


Best value: 0.012783133424818516 (params: {'learning_rate': 0.001})

